In [ ]:
import datetime
import _speed_utils as speed_utils
import _threshold_utils as threshold_utils
import altair as alt
import dask.dataframe as dd
import numpy as np
import geopandas as gpd
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
    #analysis_date,
)
from scripts import A1_sjoin_vp_segments
from shared_utils import calitp_color_palette as cp

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
CONFIG_PATH = './scripts/config.yml'

In [ ]:
STOP_SEG_DICT = helpers.get_parameters(CONFIG_PATH, "stop_segments")

In [ ]:
# Adding analysis_date here since there aren't any files for June yet
analysis_date = '2023-05-17'

### Observations (7/12)
* Step 1: Flagging
    * There are 2,704,812 rows in the dataframe original. About 10% of those rows are flagged as having zeroes in meters elapsed and seconds elapsed. 
    * There are around 4566 routes. About 57% of these routes had at least one trip with one or more rows flagged as zero.
* I took 2 passes at trying to understand why both these columns recorded zeroes.

* Step 2: `vp_pared_stops`. 
    * I grouped `vp_pared_stops` by 'shape_array_key','trip_id', and 'location_timestamp_local' OR `x` and `y`. I counted the number of unique stop sequences after grouping. If this trio had more than one unique stop sequence, that meant the timestamp or location recorded between sequences was duplicated.
    * Only around 9% of rows were flagged as having (obviously) repeated timestamps and locations. 
    * For all of these rows, both the timestamp and location were duplicated. 
    * All the routes that were flagged in step one needed a further look in step 3.
    
* Step 3: `vp_usable`
    * For one route and trip, find: all the recorded vehicle positions, sjoin of vps to segments,
    and the first and last points kept. 
    * Plot the three gdfs in a map to visually inspect what's happening.
    * Compare the sample route and trips with the trip with the highest percentage of non division by 0 rows to see what's going on. 
    
* Buckets of errors (all based on `stage0 vp`). 
    * There is only one recorded point in that segment in the raw data.
        * Fix: use the timestamp that comes after it.
        * <img src= "./speeds_images/only_one_pt_collected.png" width = 300>
    * Points are shared between segments
        * Use p20/p50/p80.
        * <img src= "./speeds_images/shared_vp.png" width = 300>
    * Points recorded are really far out and they don't touch the buffered segments.
        * Fix: figure out % of vehicle positions that are too far out.
        * <img src= "./speeds_images/dots_not_on_seg.png" width = 300>
    * No data captured for that segment at all.
        * Use p20/p50/p80. 
        * <img src= "./speeds_images/no_dots_collected.png" width = 300> 

### Merging

In [ ]:
def merge_all_speeds(analysis_date:str) -> pd.DataFrame:
    """
    Merge avg_speeds_stop_segments and
    speed_stops parquets.
    
    Args:
        date: analysis date
    """
    STG5_FILE = STOP_SEG_DICT['stage5']
    # Open up avg speeds
    avg_speeds = pd.read_parquet(f"{SEGMENT_GCS}{STG5_FILE}_{analysis_date}.parquet")
    avg_speeds = avg_speeds.drop(columns=["geometry", "district", "district_name"])
    
    # Filter  for all day flags
    avg_speeds = avg_speeds[avg_speeds.time_of_day == 'all_day'].reset_index(drop = True)
    
    STG4_FILE = STOP_SEG_DICT['stage4']
    # Open up speeds
    speeds = pd.read_parquet(f"{SEGMENT_GCS}{STG4_FILE}_{analysis_date}")
    
    # Merge
    merge_cols = ['gtfs_dataset_key','shape_array_key', 'stop_sequence']
    m1 = pd.merge(avg_speeds, speeds, on = merge_cols, how = 'inner')
    
    m1 = m1.drop_duplicates().reset_index(drop = True)
    
    return m1

In [ ]:
m1 = merge_all_speeds(analysis_date)

### Categorize

In [ ]:
def categorize_by_percentile_pandas(
    df: pd.DataFrame, 
    column_percentile: str, 
    column_str: str) -> pd.DataFrame:

    # Find percentiles
    p5 = df[column_percentile].quantile(0.05).astype(float)
    p95 = df[column_percentile].quantile(0.95).astype(float)
    
    # Keep the most extremes of both ends across the entire dataframe
    def rate(row):
        if ((row[column_percentile] >= 0) and (row[column_percentile] <= p5)):
            return f"{column_str} is low"
        elif (row[column_percentile] >= p95):
               return f"{column_str} is high"
        else:
            return f"{column_str} is avg"
    
    # Apply flags
    df[f"{column_str}cat"] = df.apply(lambda x: rate(x), axis=1)
    
    # Clean
    df[f"{column_str}cat"] = df[f"{column_str}cat"].str.replace("_", "")
    
    return df  

In [ ]:
def categorize_meters_speeds_pandas(analysis_date:str)-> pd.DataFrame:
    """
    Categorize the meters and seconds elapsed
    rows as low/high/division by 0 across the entire 
    dataframe.
    """
    start = datetime.datetime.now()
    print(start)
    
    # Load original dataframe
    df = merge_all_speeds(analysis_date)
    print(f"There are {len(df)} rows in the original dataframe") 
    
    # Categorize
    df1 = categorize_by_percentile_pandas(df, "meters_elapsed", "meters_")
    df2 = categorize_by_percentile_pandas(df1, "sec_elapsed", "sec_")
  
    # Find size of categories
    print(df2.groupby(['sec_cat','meters_cat']).size())
    
    def flag_round(row):
        if (row["meters_elapsed"] == 0) & (row["sec_elapsed"] == 0):
            return "division by 0"
        elif row["meters_cat"] == "meters is low":
            return "meters too low"
        elif row["sec_cat"] == "sec is high":
            return "seconds too high"
        else:
            return "ok"
        
    df2["flag"] = df2.apply(lambda x: flag_round(x), axis=1)
    print(df2.flag.value_counts()/len(df2)*100)
    
    end = datetime.datetime.now()
    print(f"Took {end-start}")
    return df2

In [ ]:
def keep_only_zeroes(analysis_date:str)-> pd.DataFrame:
    
    df = categorize_meters_speeds_pandas(analysis_date)
    
    # Filter out for only division by 0 
    df2 = df[(df.flag == 'division by 0')].reset_index(drop = True)
    
    # Print out some stuff of interest
    print(f"{df.trip_id.nunique()-df2.trip_id.nunique()} unique trips flagged.")
    print(f"{df2.shape_array_key.nunique()} routes flagged out of {df.shape_array_key.nunique()}.")
    print(f"{df2.shape_array_key.nunique()/df.shape_array_key.nunique() * 100} routes have 1+ row that has zeroes for meters/sec elapsed")
    print(f"{df._gtfs_dataset_name.nunique()-df2._gtfs_dataset_name.nunique()} operators are not flagged.")
    
    return df2

In [ ]:
 m2 = keep_only_zeroes(analysis_date)

In [ ]:
# m2.groupby(["loop_or_inlining"]).agg({"shape_array_key": "nunique"})

#### See how many trips for a shape ID have at least one problematic row

In [ ]:
def problematic_trips_percentage(flagged:pd.DataFrame)->pd.DataFrame:
    """
    See the % of trips for a route that has 
    at least one row that is divided by 0.
    The lower the %, the better. 
    """
    m1 = merge_all_speeds(analysis_date)
    # Number of trips that have at least one row that was divided by 0 
    # for this shape array key
    df1 = flagged.groupby(['shape_array_key']).agg({'trip_id':'nunique'}).rename(columns = {'trip_id':'trips_with_zero'}).reset_index()
    
    # Original number of trips
    df2 = m1.groupby(['shape_array_key']).agg({'trip_id':'nunique'}).rename(columns = {'trip_id':'all_trips'}).reset_index()
    
    # Merge
    m1 = pd.merge(df2, df1, how = "left", on = 'shape_array_key')
    
    # Clean
    m1['percent_of_trips_with_problematic_rows'] = (m1.trips_with_zero/m1.all_trips * 100).fillna(0)
    m1.trips_with_zero = m1.trips_with_zero.fillna(0)
    
    print(f"{len(m1[m1.percent_of_trips_with_problematic_rows == 0])/m1.shape_array_key.nunique() * 100}% of routes have 1+ division by 0 row")
    return m1

In [ ]:
trips_count = problematic_trips_percentage(m2)

In [ ]:
# trips_count.sample(5)

#### Find version of the route with the most populated info

In [ ]:
def route_most_populated(analysis_date:str)-> pd.DataFrame:
    
    flagged = categorize_meters_speeds_pandas(analysis_date)
    # First aggregation to count number of stops by flag
    agg1 = (flagged
        .groupby(['gtfs_dataset_key','shape_array_key','trip_id','flag'])
        .agg({'stop_sequence':'nunique'})
        .rename(columns = {'stop_sequence':'number_of_rows'})
        .reset_index()
       )
    
    # Create separate cols for the number of rows that are ok and rows that are division by 0
    # https://stackoverflow.com/questions/49161120/set-value-of-one-pandas-column-based-on-value-in-another-column
    agg1['division_by_zero'] = None
    agg1['ok'] = None
    agg1['division_by_zero'] = np.where(agg1.flag == 'division by 0', agg1.number_of_rows, agg1.division_by_zero)
    agg1['ok'] = np.where(agg1.flag != 'division by 0', agg1.number_of_rows, agg1.ok)
    agg1['division_by_zero'] = agg1['division_by_zero'].fillna(0)
    agg1['ok'] = agg1['ok'].fillna(0)
    
    # Aggregate again to simplify the df 
    agg1 = agg1.drop(columns = ['flag','number_of_rows'])
    agg2 = (agg1
            .groupby(['gtfs_dataset_key','shape_array_key','trip_id'])
            .agg({'division_by_zero':'sum','ok':'sum'})
            .reset_index()
           )
    
    # Find total rows for that trip
    agg2['total_rows'] = agg2.division_by_zero + agg2.ok
    
    # Find total % of rows that are ok
    agg2['percent_of_ok_rows'] = (agg2.ok/agg2.total_rows * 100)
    
    # Only keep the route/trip with the highest % of ok rows
    agg2 = (agg2
            .sort_values(['gtfs_dataset_key','shape_array_key','percent_of_ok_rows'], ascending = False)
            .drop_duplicates(['shape_array_key', 'gtfs_dataset_key'])
            .reset_index(drop = True)
           )
    
    return agg2

In [ ]:
route_most_populated_df = route_most_populated(analysis_date)

#### One function


In [ ]:
def stage_flagging(analysis_date:str)->pd.DataFrame:
    
    # Load original but merged avg speeds and speed_stop_segments
    original_df = merge_all_speeds(analysis_date)
    
    # Flag original df
    flagged_df = categorize_meters_speeds_pandas(original_df)
    
    # Rows that are only divided by 0
    zeroes_only_df = keep_only_zeroes(analysis_date)
    
    # Find % trips with 1+ row that has a division by 0 for a route
    problematic_trips_df = problematic_trips_percentage(original_df, zeroes_only_df)
    
    # Find the trip for a route that has the highest % of ok rows
    # most_populated_trip_df = route_most_populated(flagged_df)
    
    return original_df, zeroes_only_df, problematic_trips_df, most_populated_trip_df

In [ ]:
# m1, m2, trips_count, route_most_populated_df = stage_flagging(analysis_date)

### Investigate 

#### Stage3: "vp_pared_stops"/A3_loop_inlining

In [ ]:
def load_vp_stage3(flagged_df:pd.DataFrame, date:str) -> pd.DataFrame:
    """
    Load vp_pared_stops
    
    Args:
        flagged_df: df from categorize_meters_speeds_pandas()
        date: analysis date
    """
    # Subset to filter vp_pared_stops.
    shape_array_keys = flagged_df.shape_array_key.unique().tolist()
    stop_seq = flagged_df.stop_sequence.unique().tolist() 
    trip_id = flagged_df.trip_id.unique().tolist() 
    gtfs_dataset_key = flagged_df.gtfs_dataset_key.unique().tolist() 
    
    FILE = STOP_SEG_DICT['stage3']
    
    vp = pd.read_parquet(f"{SEGMENT_GCS}{FILE}_{date}",
        filters = [[('shape_array_key', "in", shape_array_keys),
                   ('stop_sequence', 'in', stop_seq), 
                   ('trip_id', 'in', trip_id), 
                   ('gtfs_dataset_key', 'in', gtfs_dataset_key)]],)
    
    # Merge to capture original df information and filter down further
    vp2 = pd.merge(flagged_df, vp, how = "inner", on = ['gtfs_dataset_key', 'trip_id','stop_sequence','shape_array_key','_gtfs_dataset_name'])
    
    return vp2

In [ ]:
def stage3_repeated_timestamps(stage3_df:pd.DataFrame)-> pd.DataFrame:
    """
    Look at how many times a time stamp is repeated a route-trip-location.
    Each of these 3 combos should have a different time for each 
    stop sequence or else the vehicle is not moving.
    
    Args:
        stage3_df: df from load_vp_stage3()
    """
    agg = (stage3_df
     .groupby(['shape_array_key','trip_id', 'location_timestamp_local'])
     .agg({'stop_sequence':'nunique'})
     .reset_index()
     .rename(columns = {'stop_sequence':'number_of_repeated_timestamps'})
    )
    
    # Only keep timestamps that are repeated more than once
    agg = (agg[agg.number_of_repeated_timestamps > 1]).reset_index(drop = True)

    return agg

In [ ]:
def stage3_repeated_locations(stage3_df:pd.DataFrame)-> pd.DataFrame:
    """
    Look at how many times a time stamp is repeated for a stop-trip-route combo.
    Each of these 3 combos should have a different location for each 
    stop sequence or else the vehicle is not changing moving.
    
    Args:
        stage3_df: df from load_vp_stage3()
    """
    # Concat x and y into a string
    stage3_df['pair'] = stage3_df.x.astype(str) + '/' + stage3_df.y.astype(str)
    
    # Count number of different stops that reference the same location
    agg = (stage3_df
     .groupby(['shape_array_key','trip_id','pair'])
     .agg({'stop_sequence':'nunique'})
     .reset_index()
     .sort_values('stop_sequence', ascending = False)
     .rename(columns = {'stop_sequence':'number_of_repeated_locs'})               
    )

    # Only keep locations that are repeated more than once
    agg = agg[agg.number_of_repeated_locs != 1].reset_index(drop = True)
    
    return agg

In [ ]:
def flag_stage3(flagged_df:pd.DataFrame, date:str) -> pd.DataFrame:
    """
    Flag the errors in stage3.
    
    Args:
        flagged_df: df from categorize_meters_speeds_pandas()
        date: analysis date
    """
    start = datetime.datetime.now()
    print(start)
    
    # Relevant rows from Vehicle Positions
    vp = load_vp_stage3(flagged_df, date)
    
    # Find repeated timestamps.
    multi_timestamps = stage3_repeated_timestamps(vp)
    
    # Find repeated locations
    multi_locs = stage3_repeated_locations(vp)
    
    # Merge
    timestamps_merge_cols = ['shape_array_key','trip_id','location_timestamp_local']
    loc_merge_cols =  ['shape_array_key','trip_id','pair']
    
    # Want everything found in vehicle positions, so do left merges
    m1 = (vp
          .merge(multi_timestamps, how="left", on= timestamps_merge_cols)
          .merge(multi_locs, how="left", on=loc_merge_cols)
         )
    
    drop_cols = ['vp_idx','x','y','hour','activity_date']
    
    # Drop columns
    m1 = m1.drop(columns = drop_cols)
    
    # Flag
    def flag(row):
        if (row["number_of_repeated_timestamps"] > 1) & (row["number_of_repeated_locs"] > 1):
            return "repeated timestamps & locations"
        elif (row["number_of_repeated_timestamps"] > 1):
            return "repeated timestamps"
        elif (row["number_of_repeated_locs"] > 1):
            return "repeated locations"
        else:
            return "check in stage 2"
        
    m1["stage3_flag"] = m1.apply(lambda x: flag(x), axis=1)
    m1 = m1.drop_duplicates().reset_index(drop = True)
    
    print(m1.stage3_flag.value_counts())
    
    check_in_stage2 = m1[m1.stage3_flag == "check in stage 2"]
    print(f"Have to check {len(check_in_stage2)/len(m1) * 100} % of rows in stage 2")
    
    end = datetime.datetime.now()
    print(f"Took {end-start}")
    return m1

In [ ]:
m3 = flag_stage3(m2, analysis_date)

In [ ]:
m3.shape_array_key.nunique()

In [ ]:
sort_by = ['_gtfs_dataset_name','shape_array_key','trip_id','stop_sequence']

In [ ]:
preview_cols = sort_by + ['stop_id','gtfs_dataset_key','location_timestamp_local','pair','stage3_flag']

In [ ]:
# m3[(m3.stage3_flag != 'check in stage 2')].sort_values(by =sort_by).sample()[preview_cols]

In [ ]:
m3[(m3.stage3_flag != 'check in stage 2')].sort_values(by =sort_by)[preview_cols].head()

#### Stage2: A1_sjoin_vp_segments


In [ ]:
def count_trips_routes(stage3_df:pd.DataFrame):
    """
    Find the route/trip combination with the 
    highest number of n_trips. 
    
    Args:
        stage3_df: df from flag_stage3() 
    """
    cols_to_keep = ['shape_array_key','_gtfs_dataset_name','gtfs_dataset_key', 'trip_id', 'n_trips']
    m1 = (stage3_df
     .sort_values(['n_trips'], ascending = False)
     .drop_duplicates(['shape_array_key'])
     [cols_to_keep]
         )
    return m1

In [ ]:
shape_trips = count_trips_routes(m3)

In [ ]:
shape_trips.head(10)

#### Look at export  file

In [ ]:
def import_stage_2(date:str, route:str, stop_sequence:str):
    FILE = STOP_SEG_DICT['stage2']
    df = pd.read_parquet(
            f"{SEGMENT_GCS}vp_sjoin/{FILE}_{date}",
            filters = [[('shape_array_key', "==", route),
                       ('stop_sequence', "==", stop_sequence)]],
        )
    return df

#### Import unique trips

In [ ]:
test_route1 = "03bc2779a66739316156b459ffc3eefa"
test_gtfs_key1 = "cdd2ad81863b6d4ad51676a1cb781ea8"
test_trip1 = "11776020"

In [ ]:
def import_unique_trips(gtfs_key:str, trip: str, route:str):
    """
    Read vp_usable file for one 
    trip/route/operator and find the unique trips.
    """
    FILE = STOP_SEG_DICT['stage1']
    vp_trips = A1_sjoin_vp_segments.add_grouping_col_to_vp(
        f"{FILE}_{analysis_date}",
        analysis_date,
       ["shape_array_key"]
    )
    
    # Filter to just one trip/route/operator
    df = vp_trips[(vp_trips.gtfs_dataset_key == gtfs_key)
                    & (vp_trips.shape_array_key == route)
                    & (vp_trips.trip_id == trip)].reset_index(drop = True)
    return df


In [ ]:
 unique_trips = import_unique_trips(test_gtfs_key1, test_trip1, test_route1)

#### Look at vehicle positions

In [ ]:
def import_vehicle_positions(unique_trips:pd.DataFrame, gtfs_key:str, trip_id:str)-> gpd.GeoDataFrame:
    """
    Create geometry form x and y for only one trip. This plots
    ALL points for the trip.
    
    Args:
        unique_trips: df from import_unique_trips()
    """
    FILE = STOP_SEG_DICT['stage0']
    
    vp = helpers.import_vehicle_positions(
            SEGMENT_GCS,
            f"{FILE}_{analysis_date}/",
            "gdf",
            filters = [[("gtfs_dataset_key", "==", gtfs_key),
                      ('trip_id', '==', trip_id)]],
            columns = ["gtfs_dataset_key", "trip_id","geometry"],
            partitioned = False
        )
    vp = vp.compute()
    """
    OLD
    FILE =  STOP_SEG_DICT['stage1']
    vp = helpers.import_vehicle_positions(
            SEGMENT_GCS,
            f"{FILE}_{analysis_date}/",
            "df",
            filters = [[("gtfs_dataset_key", "==", gtfs_key),
                      ('trip_id', '==', trip_id)]],
            columns = ["gtfs_dataset_key", "trip_id", 
                       "vp_idx", "x", "y"],
            partitioned = False
        )
    
    vp = vp.compute()
    vp = vp.merge(unique_trips, on = ["gtfs_dataset_key", "trip_id"],
            how = "inner"
        )
    
    vp_gdf = gpd.GeoDataFrame(
        vp, 
        geometry = gpd.points_from_xy(vp.x, vp.y, crs = "EPSG:4326")
    ).to_crs(PROJECT_CRS).drop(columns = ["x", "y"])
    """
    vp = vp.merge(unique_trips, on = ["gtfs_dataset_key", "trip_id"],
            how = "inner"
        )
    return vp

In [ ]:
# test = import_vehicle_positions(unique_trips,test_gtfs_key1, test_trip1)

In [ ]:
# type(test)

#### Look at segments

In [ ]:
def import_segments(flagged_df: pd.DataFrame, 
                    route:str, 
                    gtfs_key:str, 
                    trip_id:str,
                    analysis_date:str) -> gpd.GeoDataFrame:
    """
    Import cut segments and colorcode  them based on 
    whether or not it has 1+ rows that is divided by 0
    
    Args:
        flagged_df: result from df from categorize_meters_speeds_pandas()
    """
    # Load in ALL segments, flag them.
    FILE = STOP_SEG_DICT['segments_file']
    gdf = gpd.read_parquet(f"{SEGMENT_GCS}{FILE}_{analysis_date}.parquet",
                           filters = [[("shape_array_key", "==", route),
                                      ("gtfs_dataset_key", "==", gtfs_key),
                                     ]]).to_crs(PROJECT_CRS)
    
    gdf["geometry_buffered"] = gdf.geometry.buffer(35)
    gdf = gdf.set_geometry('geometry_buffered')
    
    # Distinguish between "correct" and "incorrect" seq
    # A sequence can be incorrect even if just one row is "divided by 0"
    incorrect_segments = flagged_df[
        (flagged_df.shape_array_key == route)
        & (flagged_df.gtfs_dataset_key == gtfs_key)
        & (flagged_df.trip_id == trip_id)
    ]
    incorrect_segments_list = incorrect_segments.stop_sequence.unique().tolist()
    incorrect_segments_filtered = gdf[gdf.stop_sequence.isin(incorrect_segments_list)].reset_index(drop = True)
    incorrect_segments_filtered['flag'] = 'contains 0m/0sec'
    
    # Filter for correct segments using 
    correct_segments = flagged_df[~flagged_df.stop_sequence.isin(incorrect_segments_list)]
    correct_segments_list = correct_segments.stop_sequence.unique().tolist()
    correct_segments_filtered = gdf[gdf.stop_sequence.isin(correct_segments_list)].reset_index(drop = True)
    correct_segments_filtered['flag'] = 'does not contain 0m/0sec'
    
    final = pd.concat([correct_segments_filtered, incorrect_segments_filtered])
    
    return final

#### Stops kept: last and first

In [ ]:
def find_first_last_points(route:str, trip:str, gtfs_key:str)-> gpd.GeoDataFrame:
    """
    Load gdf with only the first and last points pared. 
    """
    FILE = STOP_SEG_DICT['stage3']
    
    df = pd.read_parquet(f"{SEGMENT_GCS}{FILE}_{analysis_date}",
        filters = [[('shape_array_key', "==", route),
                  
                   ('trip_id', "==", trip), 
                   ('gtfs_dataset_key', '==', gtfs_key)]],)
    
    gdf =  gpd.GeoDataFrame(
        df, 
        geometry = gpd.points_from_xy(df.x, df.y, crs = "EPSG:4326")
    ).to_crs(PROJECT_CRS).drop(columns = ["x", "y"])
    
    gdf = gdf[['geometry','stop_sequence']]
    
    return gdf

In [ ]:
# first_last = find_first_last_points(test_route, test_trip, test_gtfs_key)

In [ ]:
# len(first_last)

#### Sjoin 

In [ ]:
def sjoin_vp_segments(segments: gpd.GeoDataFrame, vp_gdf: gpd.GeoDataFrame)-> gpd.GeoDataFrame:
    """
    Sjoin all the points with the relevant segments
    """
    vp_in_seg = gpd.sjoin(
        vp_gdf,
        segments,
        how = "inner",
        predicate = "within"
    )
    
    return vp_in_seg

#### Mapping

In [ ]:
def display_maps(all_points: gpd.GeoDataFrame, 
                 first_last_points: gpd.GeoDataFrame,
                 segments: gpd.GeoDataFrame,
                 sjoin_results: gpd.GeoDataFrame):
    """
    Create 3 maps: one to display all points, one to display
    the sjoined points, one to display first and last points that are
    automatically kept.
    
    Args:
        all_points: gdf from import_vehicle_positions()
        first_last_points: gdf from find_first_last_points()
        segments: gdf from import_segments()
        sjoin_results: gdf from sjoin_vp_segments()
    """
    segments = segments[~segments.geometry_buffered.is_empty]
    base1 = segments.explore('flag', cmap= 'tab10', height = 400, width = 600, name = 'segments')
    all_points_map = all_points.explore(m = base1, color = 'red',style_kwds = {'weight':6}, name= 'points')
    
    print('ALL POINTS')
    display(all_points_map) 
     
    # Have to use a different base geometry for sjoin 
    sjoin_points = sjoin_results.set_geometry('geometry_left')
    sjoin_segments = sjoin_results.set_geometry('geometry_right')
    sjoin_segments.geometry_right = sjoin_segments.geometry_right.buffer(35)
    base3 = sjoin_segments.explore('flag', cmap= 'tab10', height = 400, width = 600, name = 'segments')
    sjoin_map = sjoin_points.explore(m = base3, color = 'orange',style_kwds = {'weight':6},  name= 'points')
    
    print('SJOIN')
    display(sjoin_map)
    
    base2 = segments.explore('flag', cmap= 'tab10', height = 400, width = 600, name = 'segments')
    first_last_map = first_last_points.explore(m = base2, color = 'pink',style_kwds = {'weight':6},height = 400, width = 600,)
    
    print('FIRST AND LAST')
    display(first_last_map)
   

In [ ]:
# display_maps(vehicle_positions,first_last,flagged_segments)

#### Function

Previous tried routes
test_route = "106d979b9a9e6338827a8e1c145e69fd"
test_sequence = 39
test_gtfs_key = "db56b50ab86b5f7a4ae2fc2dd9889bbe"
test_trip = '1088405'

test_route2 = "0fb4f3627996269dc7075276d3b69e36"
test_gtfs_key2 = "a4f6fd5552107e05fe9743ac7cce2c55"
test_trip2 = "16939095"

test_route3 = "07c9a47264a43d8d0d16ef7109e8fd68"
test_gtfs_key3 = "db56b50ab86b5f7a4ae2fc2dd9889bbe"
test_trip3 = "1089348"

In [ ]:
def stage2_trouble_shooting(flagged_df:pd.DataFrame,
                            date:str, 
                            route:str, 
                            trip:str, 
                            gtfs_key:str,
                            analysis_date:str):
    
    """
    Putting together all the functions for stage 2
    """
    unique_trips = import_unique_trips(gtfs_key, trip, route)
    
    # Find all recorded vps
    vehicle_positions = import_vehicle_positions(unique_trips, gtfs_key, trip)
    
    # Flag segments, whether one row contains one or more 0/0 division or not
    flagged_segments = import_segments(flagged_df, route, gtfs_key, trip, analysis_date)
    
    # Find first and last pt kept
    first_last = find_first_last_points(route, trip, gtfs_key)
    
    # Sjoin 
    sjoin_results = sjoin_vp_segments(flagged_segments,vehicle_positions)
    
    # Display maps
    display_maps(vehicle_positions,first_last,flagged_segments,sjoin_results)
    

#### Example Trip 1
* Underestanding the result from `flag_stage3()`. 
* Looking at AC Transit: stop sequences 6 and 7 have different stop_ids. However, their time stamps and locations are the same. 
* It looks like they share the same point. 
* However, this isn't due to paring too many points: there just aren't enough points to choose from in the raw data.
* Also sequence 2 is extremely long.

In [ ]:
m3[(m3.stage3_flag != 'check in stage 2')].sort_values(by =sort_by)[preview_cols].head(2)

In [ ]:
def original_df_rows(original_df: pd.DataFrame, 
                     trip_id:str, 
                     shape_array_key:str) -> pd.DataFrame:
    """
    Look at the rows from the original df
    """
    preview_cols2 = ['stop_sequence','stop_id','meters_elapsed','sec_elapsed',]
    
    df = (original_df[(original_df.trip_id == trip_id)
            & (original_df.shape_array_key == shape_array_key)]
            [preview_cols2]
            .sort_values(['stop_sequence'])
           )
            
    return df

In [ ]:
test_route1 = "03bc2779a66739316156b459ffc3eefa"
test_gtfs_key1 = "cdd2ad81863b6d4ad51676a1cb781ea8"
test_trip1 = "11776020"

In [ ]:
# % of trips with problematic rows for this route
trips_count[trips_count.shape_array_key == test_route1]

In [ ]:
original_df_rows(m1, test_trip1, test_route1)

In [ ]:
stage2_trouble_shooting(flagged_df= m3,
                        date = analysis_date,
                        route = test_route1,
                        trip = test_trip1,
                        gtfs_key = test_gtfs_key1,
                        analysis_date = analysis_date)

#### Example Trip 2
* Underestanding the result from `flag_stage3()`. 
* Same thing as Ex Trip 1 except kooking at LA Metro: stop sequences 45 and 46 have different stop_ids. However, their time stamps and locations are the same. 
* Same issue: segments 45 and 46 are sharing points. There aren't enough points captured.
* In general, it looks like this route doesn't have a lot of rows that are ok.

In [ ]:
test_route2 = "38c814829dff816aa87c606c3aab4f45"
test_gtfs_key2 = "65d9589130415c685b89f4f7c2d8bd7e"
test_trip2 = "10294000051654-DEC22"

In [ ]:
# Original number of rows for this trip
len(m1[(m1.trip_id == test_trip2) & (m1.shape_array_key == test_route2)])

In [ ]:
# Rows with zeroes...a lot of them.
len(m3[(m3.trip_id == test_trip2) & (m3.shape_array_key == test_route2)])

In [ ]:
m3[(m3.stage3_flag != 'check in stage 2') & (m3.shape_array_key == test_route2) 
   & (m3.stop_sequence.isin([45,46]))].sort_values(by =sort_by)[preview_cols]

In [ ]:
# % of trips with problematic rows for this route
trips_count[trips_count.shape_array_key == test_route2]

In [ ]:
original_df_rows(m1, test_trip2, test_route2)

In [ ]:
stage2_trouble_shooting(flagged_df= m3,
                        date = analysis_date,
                        route = test_route2,
                        trip = test_trip2,
                        gtfs_key = test_gtfs_key2,
                        analysis_date = analysis_date)

In [ ]:
# See which trip has the most rows that are ok for this route
route_most_populated_df[route_most_populated_df.shape_array_key == test_route2]

In [ ]:
# Plot the trip from this route with the highest % of ok rows
stage2_trouble_shooting(flagged_df= m3,
                        date = analysis_date,
                        route = test_route2,
                        trip = "10294000051724-DEC22",
                        gtfs_key = test_gtfs_key2,
                        analysis_date = analysis_date)

#### Example Trip 3
* Choosing a route/trip with high n_trips that isn't Muni or LA Metro to shake things up.
* San Diego Vehicle Positions
* Segments 44 and 45 don't have any rows in the original dataframe for trip 16938440.

In [ ]:
test_route3 = "1fc55d9df0cd785dddc864bf1b72976f"
test_gtfs_key3 = "a4f6fd5552107e05fe9743ac7cce2c55"
test_trip3 = "16938440"

In [ ]:
# m1[(m1.stop_sequence == 38) & (m1.shape_array_key == test_route3)]

In [ ]:
original_df_rows(m1, test_trip3, test_route3)

In [ ]:
stage2_trouble_shooting(flagged_df= m3,
                        date = analysis_date,
                        route = test_route3,
                        trip = test_trip3,
                        gtfs_key = test_gtfs_key3,
                        analysis_date = analysis_date)

In [ ]:
# Trip from this route with the most ok rows
route_most_populated_df[route_most_populated_df.shape_array_key == test_route3]

In [ ]:
# Test the trip with the most ok rows
stage2_trouble_shooting(flagged_df= m3,
                        date = analysis_date,
                        route = test_route3,
                        trip = "16938341",
                        gtfs_key = test_gtfs_key3,
                        analysis_date = analysis_date)

In [ ]:
original_df_rows(m1, "16938341", test_route3)